In [ ]:
!pip install rasterio geopandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 72.2 MB/s eta 0:00:00:00:0100:01


In [2]:
import rasterio
import numpy as np
import geopandas as gpd

In [3]:
file_path = "/kaggle/input/usa-tiff-file/Ras_74_terrain22_by_Gcluster15_Sinks.tif"

# "mountains", "plain", "plateau", "valley", "cliff", "highlands", "hills",
# classes = {
#     1: "plain",
#     2: "mountains",
#     3: "cliff",
#     4: "highlands",
#     5: "highlands",
#     6: "plateau",
#     7: "plateau",
#     8: "valley",
#     9: "plateau",
#     10: "hills",
#     11: "plateau",
#     12: "cliff",
#     13: "hills",
#     14: "valley",
#     15: "plain",
# }

classes = {
    1: "mountains",
    2: "cliffs",
    3: "hills",
    4: "cliffs",
    5: "highlands",
    6: "highlands",
    7: "valleys",
    8: "hills",
    9: "plateaus",
    10: "plateaus",
    11: "plateaus",
    12: "plateaus",
    13: "valleys",
    14: "valleys",
    15: "plateaus",
    16: "plateaus",
    17: "plateaus",
    18: "plateaus",
    19: "plains",
    20: "plains",
    21: "plains",
    22: "plains"
}


In [4]:
with rasterio.open(file_path) as src:
    terrain_band = src.read(1)
    metadata = src.meta.copy()
    transform = src.transform

metadata

{'driver': 'GTiff',
 'dtype': 'uint8',
 'nodata': 0.0,
 'width': 43317,
 'height': 25286,
 'count': 1,
 'crs': CRS.from_wkt('GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]'),
 'transform': Affine(0.000833333333, 0.0, -113.93710915290283,
        0.0, -0.000833333333, 50.00003948755602)}

In [ ]:
rows, cols = terrain_band.shape
rows, cols

(25286, 43317)

In [8]:
from shapely.geometry import box

In [10]:
regions = {terrain: [] for terrain in classes.values()}
comp_region = np.vectorize(classes.get)(terrain_band)             # Converting the region to class names
visited = np.full_like(a=terrain_band, fill_value=0, dtype=np.uint8)
visited[:10]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [ ]:

# for class_id, class_name in classes.items():

region_size = 32
# print("Checking for {} terrain for region size: {}".format(class_name, region_size))

for i in range(rows-region_size):
    for j in range(cols-region_size):

        region = comp_region[i:i+region_size, j:j+region_size]     # Creating a region of required size for comparison
        visited_check_region = visited[i:i+region_size, j:j+region_size]       # Creating a sub-section of visited array for comparison

        curr_class = region[0,0]
        if all(len(regions[terr])>100 for terr in regions.keys()):
          break
        if curr_class == None or len(regions[curr_class])>100:
          continue
        if np.all(visited_check_region == 0) and np.all(region == curr_class):
          print("Region is valid. Extracting World Coordinates")
          visited[i:i+region_size, j:j+region_size] = 1

          # Computing world coordinates from indices
          min_lat, min_lon = transform * (j, i)
          max_lat, max_lon = transform * (j+region_size, i+region_size)
          coord_box = box(min_lat, min_lon, max_lat, max_lon)

          print("Storing #{} in class {} in bouding region: {}" .format(len(regions[curr_class]), curr_class, coord_box))
          regions[curr_class].append(coord_box)




Region is valid. Extracting World Coordinates
Storing #0 in class plateaus in bouding region: POLYGON ((-107.71544248872483 49.999206154223025, -107.71544248872483 49.97170615423402, -107.74294248871384 49.97170615423402, -107.74294248871384 49.999206154223025, -107.71544248872483 49.999206154223025))
Region is valid. Extracting World Coordinates
Storing #1 in class plateaus in bouding region: POLYGON ((-107.55377582212283 49.999206154223025, -107.55377582212283 49.97170615423402, -107.58127582211183 49.97170615423402, -107.58127582211183 49.999206154223025, -107.55377582212283 49.999206154223025))
Region is valid. Extracting World Coordinates
Storing #2 in class plateaus in bouding region: POLYGON ((-107.48710915548284 49.999206154223025, -107.48710915548284 49.97170615423402, -107.51460915547183 49.97170615423402, -107.51460915547183 49.999206154223025, -107.48710915548284 49.999206154223025))
Region is valid. Extracting World Coordinates
Storing #3 in class plateaus in bouding regio

In [12]:
for label, box in regions.items():
    gdf = gpd.GeoDataFrame(geometry=box, crs="EPSG:4326")
    gdf.to_file(f"{label}.geojson", driver="GeoJSON")
    print(f"Extracted {len(regions[label])} regions for terrain class {label}.")

Extracted 101 regions for terrain class mountains.
Extracted 101 regions for terrain class cliffs.
Extracted 101 regions for terrain class hills.
Extracted 101 regions for terrain class highlands.
Extracted 101 regions for terrain class valleys.
Extracted 101 regions for terrain class plateaus.
Extracted 101 regions for terrain class plains.
